<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Lesson 14 - Latent Variables and Natural Language Processing

---


## Activity: Twitter Lab
In this exercise, we will compare some of the classical NLP tools from the last class with these more modern latent variable techniques.  We will do this by comparing information extraction on Twitter using two different methods.

There is a pre-existing file of captured tweets you can use.  It is located in the datasets folder of the class repo. 

The sample `captured-tweets.txt` dataset in the repo was generated by collecting ~5000 tweets from the TwitterAPI using the keywords:
- `Google`
- `Microsoft`
- `Goldman Sachs`
- `Citigroup`
- `Tesla`
- `Verizon`
- `Syria`
- `Iran`
- `Israel`
- `Iraq`

In [1]:
# Unicode Handling
from __future__ import unicode_literals
import codecs

import numpy as np
import gensim

# spacy is used for pre-processing and traditional NLP
import spacy
from spacy.en import English

nlp_toolkit = spacy.load('en')

# Gensim is used for LDA and word2vec
from gensim.models.word2vec import Word2Vec

C:\Users\Ayham\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [8]:
# Loading the tweet data
filename = 'dataset/captured-tweets.txt'
tweets = []
for tweet in codecs.open(filename, 'r', encoding="utf-8"):
    tweets.append(tweet)

### Example for nlp_toolkit

In [9]:
doc = nlp_toolkit(u'London is a big city in the United Kingdom.')
for ent in doc.ents:
    print(ent.label_, ent.text)
    # GPE London
    # GPE United Kingdom

GPE London
GPE the United Kingdom


## Exercise 1a

Write a function that can take a sentence parsed by `spacy` and identify if it mentions a company named 'Google'. Remember, `spacy` can find entities and codes them as `ORG` if they are a company. Look at the slides for class 13 if you need a hint.

### Bonus (1b)

Parameterise the company name so that the function works for any company.

In [10]:
def mentions_company(parsed):
    for entity in parsed.ents:
        if entity.text == "Google" and entity.label_ == 'ORG':
            return True
    return False

# 1b

def mentions_company(parsed, company='Google'):
    for entity in parsed.ents:
        if entity.text == company and entity.label_ == 'ORG':
            return True
    return False

## Exercise 1c

Write a function that can take a sentence parsed by `spacy` 
and return the verbs of the sentence (preferably lemmatized)

In [11]:
def get_actions(parsed):
    actions = []
    for el in parsed:
        if el.pos == spacy.parts_of_speech.VERB:
            actions.append(el.text)
    return actions

## Exercise 1d
For each tweet, parse it using spacy and print it out if the tweet has 'Google' as company and 'spying' or 'announce' as a verb. You'll need to use your `mentions_company` and `get_actions` functions.

In [16]:
for tweet in tweets:
    parsed = nlp_toolkit(tweet)
    if mentions_company(parsed, 'Google'):
        actions = get_actions(parsed)        
        if 'spying' in actions or 'announce' in actions:
            print(tweet)

Microsoft joins Google and Facebook with warnings to users about government spying https://t.co/tXjvgeHFOu https://t.co/TW7aJb37q9



In [17]:
actions

['joins', 'spying']

In [18]:
for tweet in tweets:
    parsed = nlp_toolkit(tweet)
    if mentions_company(parsed, 'Google'):
        actions = get_actions(parsed)
        if 'release' in actions or 'announce' in actions or 'building' in actions:
            print(tweet)

RT @business: Where Amazon, Microsoft, Google, IBM and DigitalOcean are building data centers https://t.co/VX047Jm9tq https://t.co/opTaZzO7…



## Exercise 1e
Write a function that identifies countries - HINT: the entity label for countries is GPE (or GeoPolitical Entity)



In [19]:
def mentions_country(parsed, country):
    for entity in parsed.ents:
        if entity.text == country and entity.label_ == 'GPE':
            return True
    return False

## Exercise 1f

Re-run (d) to find country tweets that discuss 'Iran' announcing or releasing.


In [20]:
for tweet in tweets:
    parsed = nlp_toolkit(tweet)

    if mentions_country(parsed, 'Iran'):
        actions = get_actions(parsed)
        if 'release' in actions or 'announce' in actions:
            print(tweet)

RT @cerenomri: "Literally every US ally in Mideast is on brink of hot war w/ Iran, so we're going to release $100 billion to Iran this mont…

GOBE! Iran warns Nigeria to release Shiite leader El-Zakzaky - SEE https://t.co/TRshnC6sVU

GOBE! Iran warns Nigeria to release Shiite leader El-Zakzaky - SEE https://t.co/SlvcQtk3vE

RT @cerenomri: "Literally every US ally in Mideast is on brink of hot war w/ Iran, so we're going to release $100 billion to Iran this mont…

Hhmmm. Iran claiming to have 'warned Nigeria' to release detained Shiite leader.... @afalli

RT @cerenomri: "Literally every US ally in Mideast is on brink of hot war w/ Iran, so we're going to release $100 billion to Iran this mont…



In [21]:
actions

[]

## Exercise 2
Build a word2vec model of the tweets we have collected using gensim.
First take the collection of tweets and tokenize them using spacy.

### Exercise 2a:
* Think about how this should be done. 
* Should you only use upper-case or lower-case? 
* Should you remove punctuations or symbols? 

In [22]:
text_split = [[x.text if x.pos != spacy.parts_of_speech.VERB else x.lemma_ 
                for x in nlp_toolkit(t)] for t in tweets]

In [23]:
tweets[0]

'I made a(n) Small Tourmaline in Paradise Island! https://t.co/cAoW1b6DRc #Gameinsight #Androidgames #Android\n'

In [24]:
text_split[:1]

[['I',
  'make',
  'a(n',
  ')',
  'Small',
  'Tourmaline',
  'in',
  'Paradise',
  'Island',
  '!',
  'https://t.co/cAoW1b6DRc',
  '#',
  'Gameinsight',
  '#',
  'Androidgames',
  '#',
  'Android',
  '\n']]

### Exercise 2b:
Build a word2vec model.
Test the window size as well - this is how many surrounding words need to be used to model a word. What do you think is appropriate for Twitter? 

In [25]:
model = Word2Vec(text_split, size=100, window=4, min_count=5, workers=4)

### Exercise 2c:
Test your word2vec model with a few similarity functions. 
* Find words similar to 'tweet'.
* Find words similar to 'rank'.
* Find words similar to 'Google'.
* Find words similar to 'Syria'. 



In [26]:
model.wv.most_similar(positive=['tweet'],topn=10)

[('10', 0.999434769153595),
 ('help', 0.9993937611579895),
 ('join', 0.9993391633033752),
 ('could', 0.9993309378623962),
 ('say', 0.9993267059326172),
 ('every', 0.9993239641189575),
 ('Rouhani', 0.9993118047714233),
 ('please', 0.9993070960044861),
 ('US', 0.9993057250976562),
 ('3', 0.9992966055870056)]

In [27]:
model.wv.most_similar(positive=['rank'],topn=10)

[('partners', 0.9965410232543945),
 ('refuse', 0.9965380430221558),
 ('The', 0.9965197443962097),
 ('NEW', 0.9964765310287476),
 ('Day', 0.996303379535675),
 ('In', 0.9962840676307678),
 ('Syrian', 0.9962807297706604),
 ('No', 0.9962160587310791),
 ('Tesla', 0.9962055087089539),
 ('How', 0.9961835741996765)]

In [28]:
model.wv.most_similar(positive=['Google'],topn=10)

[('Play', 0.9933983683586121),
 ('ROYALTY', 0.9928666949272156),
 ('famous', 0.9915315508842468),
 ('juice', 0.9911171197891235),
 ('RADIO', 0.9887184500694275),
 ('https://t.co/6Y96MXdAOD', 0.988685667514801),
 ('@BrookingsInst', 0.9885120987892151),
 ('S', 0.9880359768867493),
 ('Play：https://t.co', 0.9875374436378479),
 ('APPS', 0.9874345064163208)]

In [29]:
model.wv.most_similar(positive=['Syria'],topn=12)

[('opposition', 0.999240517616272),
 ('Russia', 0.9984389543533325),
 ('death', 0.9973208904266357),
 ('Arab', 0.9973099231719971),
 ('Ads', 0.9971376657485962),
 ('0161', 0.9969374537467957),
 ('in', 0.9969058632850647),
 ('conference', 0.9967876672744751),
 ('+', 0.9966884851455688),
 ('art', 0.9966536164283752),
 ('human', 0.996423602104187),
 ('benefit', 0.9963943958282471)]

# Exercise 2d

Adjust the choices in (b) and (c) as necessary.


## Exercise 3

Filter tweets to those that mention 'Iran' or similar entities and 'plan' or similar entities.
* Do this using just spacy.
* Do this using word2vec similarity scores.

In [30]:
# Using word2vec similarity scores
def tweet_sim(word_1,word_2,threshold_1=0.99,threshold_2=0.99):
    tweet_list = []
    i = 0
    for tweet in tweets[:200]:
        tweet_sublist = []
        parsed = nlp_toolkit(tweet)

        similarity_to_1 = max([model.similarity(word_1, tok.text) for tok in parsed if tok.text in model.wv.vocab])
        similarity_to_2 = max([model.similarity(word_2, tok.text) for tok in parsed if tok.text in model.wv.vocab])
        if similarity_to_1 > threshold_1 and similarity_to_2 > threshold_2:
            tweet_sublist.append(i)
            tweet_sublist.append([similarity_to_1, similarity_to_2])
            tweet_sublist.append(tweet)
            tweet_list.append(tweet_sublist)
            i += 1
        
    print('tweets above threshold:', i)
    return tweet_list

In [31]:
list_google_find = tweet_sim('Google','find',0.99,0.995)

tweets above threshold: 58


In [32]:
[lg[2] for lg in list_google_find][:10]

['Claim your Google Play Gift Card Code... https://t.co/ySYH1x5kQl #amazon #itunes #googl… https://t.co/ayDI4X1FKO\n',
 "I've entered to win a Google Nexus 6P from  !    https://t.co/4vFHfhaBey\n",
 "RT @kamcb29: I've entered to win a Google Nexus 6P from @MakeUseOf ! https://t.co/o30B9xG6Dx #giveaway #competition\n",
 'I LOVE your Google plus page with the other girls! 💜😆\n',
 "After I've Google &amp; read a ton of articles on the same subject, I remember, I could've just searched YouTube for this shit 😠\n",
 'RT @ShowerThoughtts: Apple has "air", Amazon has "Fire", Google has "earth", why doesn\'t Microsoft have "water"?\n',
 "-Looks up on Google 'MikexJeremy' secretly- &lt;33 ;) [@FnafSchimdt,@MikeSchmit10,]#SenpaiBot~\n",
 'RT @_silentbent_: Go support @dadeputy single "It\'s Okay" feat @jaesongreen on iTunes,Google… https://t.co/WPc1jwFLs0\n',
 'Ever wanted to become a Google Small Business Advisor? Now you can! https://t.co/fcOkq6srSX\n',
 'Top Android Apps (without all the games

In [33]:
list_iran_plan = tweet_sim('Iran','plan',0.99,0.995)

tweets above threshold: 11


In [34]:
[lg[2] for lg in list_iran_plan][:10]

['RT https://t.co/HqV9jSb6In Android application related to google maps -- 2 by nsatheeshk: Need to update the Android Mobil... …\n',
 'LebanonHashtag: RT probrandz: #Facebook Open Sources Artificial #Intelligence Servers Before #Google | Re/code https://t.co/f8RTJxEzSx\n',
 'KalGuntuku: RT bunnymiilk "RT 30seclovelive: [RT to win!] 30 Seconds of Love Live! 5k Followers Giveaway #1: $25 iTunes/Google Play gift ca…\n',
 '[Minecraft] I google "Natural arch" and a Minecraft image popped up for a real geographical scenic spot by Skandal… https://t.co/9ZZrZrfJpP\n',
 'LebanonHashtag: RT probrandz: #Facebook Open Sources Artificial #Intelligence Servers Before #Google | Re/code https://t.co/VgWBxAG52q\n',
 'Ford adds Apple CarPlay and Android Auto to Sync 3,... https://t.co/NSPCS9KHWE #google | https://t.co/MZBQSll3dP https://t.co/tHKibTMDlU\n',
 "RT @dinowoowife: [MY] JIMIN 2016 'RISE' stock sale by MINingful moment @miningfulmoment ♥ https://t.co/DwRaGK45D3 https://t.co/Nh04n1sB5b\n",
 "RT